## Day 9: Rope Bridge

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day09/puzzle.ipynb)

Actually, the input can be a valid F# code.
```fsharp
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
```
All we have to do is save the input as fsx file, define corresponding function and then load the input so that it will execute.

The functions are going to mutate some global state. For better control, we can define the actual logic functions in an immutable manner which can be called by functions `R`, `U`, `L` and `D`.

The `Point` type is defined in the `common.fsx` script. Effectively it is a wrapper for `int*int`, but also provides `(+)` and `(-)` operators. Generally speaking, it is both a Point and a Vector, depending on the context.

In [136]:
#load "../common/common.fsx"

type State = { Knots : Point list; }
with static member DefaultPart1 = { Knots = [ Point(0,0); Point(0,0) ] }
     static member DefaultPart2 = { Knots = List.replicate 10 (Point(0,0)) }

This is going to be handy to simulate all the movements as some state changes at each steps. But it is important to make sure that each motion produces a sequence of states. 

In [137]:
let rec moveTail (knots : Point list) = 
    match knots with
    | [] -> []
    | [x] -> [x]
    | head::x::tail -> 
        let (Point(dx, dy)) = head - x
        if (abs dx > 1 || abs dy > 1)
        then head :: moveTail((x + Point(sign dx, sign dy)) :: tail)
        else knots

let move' (Point(x,y)) (state : State) =
    let head::tail = state.Knots
    let newHead = head + Point(x,y)
    { Knots = moveTail (newHead::tail) }
    

let moveMany' v count state = 
    (List.replicate count v, state)
    ||> List.mapFoldBack (fun step prevState -> let s = move' step prevState in s,s) 
    |> fst
let R' steps state = 
    moveMany' (Point(0,1)) steps state
let L' steps state = 
    moveMany' (Point(0,-1)) steps state
let U' steps state = 
    moveMany' (Point(1,0)) steps state
let D' steps state = 
    moveMany' (Point(-1,0)) steps state

Define the global mutable list of states and mutating functions

In [138]:
let mutable states = []
let R steps = states <- (R' steps (List.head states)) @ states
let L steps = states <- (L' steps (List.head states)) @ states
let U steps = states <- (U' steps (List.head states)) @ states
let D steps = states <- (D' steps (List.head states)) @ states

In [139]:
#load "../common/matrixFormatting.fsx"
let pointsToDisplayable (points : Point array) = 
    let maxi = points |> Array.map (fun (Point(x,_)) -> x) |> Array.max
    let maxj = points |> Array.map (fun (Point(_,y)) -> y) |> Array.max
    let mini = points |> Array.map (fun (Point(x,_)) -> x) |> Array.min
    let minj = points |> Array.map (fun (Point(_,y)) -> y) |> Array.min
    let matrix = Array2D.createBased mini minj (maxi - mini + 1) (maxj - minj + 1) System.Drawing.Color.Transparent
    points |> Array.iter (fun (Point(x,y)) -> matrix[mini+maxi-x(*mirror*),y] <- System.Drawing.Color.Red)
    matrix |> Array2D.rebase |> toDisplayable id

### Part 1

With these functions defined, we can now execute the sample input and process the states history.

In [140]:
states <- [ State.DefaultPart1 ]
#load "data_sample1.fsx"
let tailTrail s = 
    s |> List.toArray |> Array.map (fun s -> List.last s.Knots) |> Array.distinct
states |> tailTrail |> Array.length |> display
states |> tailTrail |> pointsToDisplayable

13

For the actual input we are about to reset the state first:

In [141]:
states <- [ State.DefaultPart1 ]
#load "data_actual.fsx"

states |> tailTrail |> Array.length |> display
states |> tailTrail |> pointsToDisplayable |> withSettings { Width = 700 }

6384

### Part 2

In [142]:
states <- [ State.DefaultPart2 ]
#load "data_sample2.fsx"

states |> tailTrail |> Array.length |> display
states |> tailTrail |> pointsToDisplayable

36

In [143]:
states <- [ State.DefaultPart2 ]
#load "data_actual.fsx"

states |> tailTrail |> Array.length |> display
states |> tailTrail |> pointsToDisplayable |> withSettings { Width = 700 }

2734

It is interensting, how the trail from all the knots is going to look like

In [144]:
let knotsTrail s = 
    s 
    |> Seq.collect (fun s -> s.Knots) 
    |> Seq.distinct
    |> Seq.toArray


states |> knotsTrail |> pointsToDisplayable |> withSettings { Width = 700 }